# Biostat 257 Homework 3

**Due May 14 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


## Q1. Big $n$ linear regression

People often think linear regression on a dataset with millions of observations is a big data problem. Now we learnt various methods for solving linear regression and should realize that, with right choice of algorithm, it is a problem that can be handled by any moderate computer.

### Q1.1 Download data (10pts)

Download the flight data from <https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7>. **Do not put data files in Git.** You will lose points if you do. For grading purpose (reproducibility), we will assume that data files are in a subfolder `flights`.

In [2]:
;ls -l flights

total 3095696
-rw-rw-r--@ 1 minsookim  staff   12652442 May  5 04:33 1987.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   49499025 May  5 04:33 1988.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   49202298 May  5 04:33 1989.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   52041322 May  5 04:33 1990.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   49877448 May  5 04:33 1991.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   50040946 May  5 04:33 1992.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   50111774 May  5 04:33 1993.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   51123887 May  5 04:33 1994.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   74881752 May  5 04:33 1995.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   75887707 May  5 04:33 1996.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   76705687 May  5 04:34 1997.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   76683506 May  5 04:34 1998.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   79449438 May  5 04:34 1999.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   82537924 May  5 04:34 2000.csv.bz2
-rw-rw-r--@ 1 minsookim  staff   8

Find out how many data points are in each year.

In [3]:
using CodecBzip2
using CSV
using DataFrames
using Distributions
using LinearAlgebra
using Serialization
using StatsModels
using SweepOperator
ENV["COLUMNS"] = 200;

In [4]:
# print first 10 lines of 1987 data
run(pipeline(`bunzip2 -c flights/1987.csv.bz2`, `head`))

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,NA,23,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,NA,14,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,NA,29,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,NA,-2,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,NA,33,19,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1987,10,21,3,728,730,848,849,PS,1451,NA,80,79,NA,-1,-2,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1987,10,22,4,728,730,852,849,PS,1451,NA,84,79,NA,3,-2,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1987,10,23,5,731,730,902,849,PS,1451

Base.ProcessChain(Base.Process[Process(`bunzip2 -c flights/1987.csv.bz2`, ProcessSignaled(13)), Process(`head`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

In [5]:
# count number of data points
total = 0 
for i in 1987:2008
    file = "flights/$i.csv.bz2"
    open(file, "r") do io
        n = countlines(Bzip2DecompressorStream(io))
        println(string(file, ": ", n))
        global total += n
    end
end
println("total number of lines: $total")

flights/1987.csv.bz2: 1311827
flights/1988.csv.bz2: 5202097
flights/1989.csv.bz2: 5041201
flights/1990.csv.bz2: 5270894
flights/1991.csv.bz2: 5076926
flights/1992.csv.bz2: 5092158
flights/1993.csv.bz2: 5070502
flights/1994.csv.bz2: 5180049
flights/1995.csv.bz2: 5327436
flights/1996.csv.bz2: 5351984
flights/1997.csv.bz2: 5411844
flights/1998.csv.bz2: 5384722
flights/1999.csv.bz2: 5527885
flights/2000.csv.bz2: 5683048
flights/2001.csv.bz2: 5967781
flights/2002.csv.bz2: 5271360
flights/2003.csv.bz2: 6488541
flights/2004.csv.bz2: 7129271
flights/2005.csv.bz2: 7140597
flights/2006.csv.bz2: 7141923
flights/2007.csv.bz2: 7453216
flights/2008.csv.bz2: 2389218
total number of lines: 118914480


In [6]:
# # figure out which airlines appear in each year of 1987-2008
# airlines = Vector{Vector{String}}(undef, 22)
# @time for year in 1987:2008
#     println("year $year")
#     filename = "flights/" * string(year) * ".csv.bz2"
#     df = open(filename, "r") do io
#         CSV.File(
#             Bzip2DecompressorStream(io),
#             select = ["UniqueCarrier"],
#             types = Dict("UniqueCarrier" => String),
#             missingstring = "NA"
#         ) |> DataFrame
#     end
#     airlines[year - 1986] = unique(df[!, :UniqueCarrier])
# end
# intersect(airlines...) |> sort

### Q1.2 (10 pts) Problem size

We are interested in how the arrival delay of a flight, `ArrDelay`, depends on the distance traveled (`Distance`), departure delay (`DepDelay`), weekday (`DayOfWeek`), and airline (`UniqueCarrier`). 

We want to fit a linear regression `ArrDelay ~ 1 + Distance + DepDelay + DayOfWeek + UniqueCarrier` using data from 1987-2008. Treat `DayOfWeek` as a factor with 6 levels. We use the dummy coding with `1` (Monday) as the base level. Treat `UniqueCarrier` as a factor with 8 levels: "AA", "AS", "CO", "DL", "NW", "UA", "US", and "WN". We use the dummy coding with "AA" as the base level.

Will the design matrix $\mathbf{X}$ (in double precision) fit into the memory of your computer? 

The data in total has ~119 million rows and 16 columns, so using double presion (8 bytes) will yield $119 \cdot 10^6 \times 16 \times 8 \approx 15 \cdot 10^9$ bytes of memory. My laptop has 32 GB memory, so around half of it will be used in managing this data. 

### Q1.3 (30 pts) Choose algorithm

Assume your computer has limited memory, say only 1GB. Review the [Summary of Linear Regression](https://ucla-biostat-257-2021spring.github.io/slides/15-linreg/linreg.html) and choose one method in the table to solve the linear regression.

Report the estimated regression coefficients $\widehat \beta$, estimated variance $\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - p)$, and standard errors of $\widehat \beta$.

Hint: It took my laptop about 10-11 minutes to import data and fit linear regression.

### Sweep operator to estimate $\widehat \beta$, $\widehat \sigma^2$, and $\text{se}(\widehat \beta)$

In [7]:
# initialize arrays
XᵗXsum = zeros(16, 16)
Xᵗysum = zeros(16)
yᵗysum = 0 
n = 0

for i in 1987:2008
    file = "flights/$i.csv.bz2"
    println("processing $file")
    
    # load data into DataFrame
    df = open(file, "r") do io
        CSV.File(
            Bzip2DecompressorStream(io), 
            select = ["DayOfWeek", "UniqueCarrier", "ArrDelay", 
                "DepDelay", "Distance"],
            types = Dict(
                "DayOfWeek" => UInt8,
                "UniqueCarrier" => String, 
                "ArrDelay" => Float64, 
                "DepDelay" => Float64, 
                "Distance" => UInt16
                ),
            missingstring = "NA"
            ) |> DataFrame
    end

    # drop rows with missing values
    dropmissing!(df)
    
    # filter out rows not in the airline list
    airlines = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]
    filter!(row -> row[:UniqueCarrier] ∈ airlines, df)
    
    # model frame
    mf = ModelFrame(
        @formula(ArrDelay ~ 1 + DayOfWeek + Distance + DepDelay + UniqueCarrier), 
        df,
        contrasts = Dict(
            :DayOfWeek => StatsModels.DummyCoding(base = 1, levels = UInt8.(1:7)),
            :UniqueCarrier => StatsModels.DummyCoding(
                base = "AA", 
                levels = ["AA", "AS", "CO", "DL", "NW", "UA", "US", "WN"]
            )
        )
    )
    
    # generate the covariate matrix X
    X = modelmatrix(mf)
    
    # generate the response vector Y
    y = df[!, :ArrDelay]
    
    # compute XᵗX, Xᵗy, and yᵗy
    n += size(X, 1)
    BLAS.syrk!('U', 'T', 1.0, X, 1.0, XᵗXsum)
    BLAS.gemv!('T', 1.0, X, y, 1.0, Xᵗysum)
    yᵗysum += dot(y, y)
end

# sweep operator
G = [XᵗXsum Xᵗysum; Xᵗysum' yᵗysum]
sweep!(G, 1:16)
β̂ = G[1:16, end]
σ̂² = G[end, end] / (n - 16)
varβ̂ = - diag(G[1:16, 1:16]) * σ̂²;

processing flights/1987.csv.bz2
processing flights/1988.csv.bz2
processing flights/1989.csv.bz2
processing flights/1990.csv.bz2
processing flights/1991.csv.bz2
processing flights/1992.csv.bz2
processing flights/1993.csv.bz2
processing flights/1994.csv.bz2
processing flights/1995.csv.bz2
processing flights/1996.csv.bz2
processing flights/1997.csv.bz2
processing flights/1998.csv.bz2
processing flights/1999.csv.bz2
processing flights/2000.csv.bz2
processing flights/2001.csv.bz2
processing flights/2002.csv.bz2
processing flights/2003.csv.bz2
processing flights/2004.csv.bz2
processing flights/2005.csv.bz2
processing flights/2006.csv.bz2
processing flights/2007.csv.bz2
processing flights/2008.csv.bz2


In [8]:
σ̂²

230.48901403675245

In [9]:
[β̂ varβ̂]

16×2 Matrix{Float64}:
  0.83561     3.87466e-5
  0.259139    3.43987e-5
  0.668682    3.43671e-5
  1.01573     3.44149e-5
  0.812092    3.44055e-5
 -1.6118      3.66607e-5
 -0.704679    3.52617e-5
 -0.00140991  8.61583e-12
  0.924935    3.36286e-9
  1.03988     0.000102511
  0.25794     4.57601e-5
  0.662151    3.12021e-5
  0.0340912   4.01901e-5
  0.0402917   3.44179e-5
 -0.452353    3.45962e-5
 -2.976       3.33723e-5

### Q1.4 Be proud of yourself

Go to your resume/cv and claim you have experience performing analytics on data with 100 million observations.

## Q2. PageRank

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

### Q2.1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

Suppose $\boldsymbol{x}, \boldsymbol{y} \in \mathbb{R}^{n \times 1}$ are defined in such a way that

$$
\begin{eqnarray*}
	x_{i}= \begin{cases}
	\frac{p}{r_i} & r_i > 0 \\
	0 & r_i = 0
	\end{cases}
\end{eqnarray*}
$$

$$
\begin{eqnarray*}
	y_{i}= \begin{cases}
	\frac{1 - p}{n} & r_i > 0 \\
	\frac{1}{n} & r_i = 0
	\end{cases}
\end{eqnarray*}
$$

Then the transition matrix $\mathbf{P}$ can be formulated as

$$
\mathbf{P} = \text{diag}(\boldsymbol{x}) \cdot \mathbf{A}  + \text{diag}(\boldsymbol{y}) \cdot \mathbf{1}_n\mathbf{1}_n^T
$$

### Q2.2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

### Q2.3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [10]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md);

include group.jl for user defined matrix generators
verify download of index files...
reading database
adding metadata...
adding svd data...
writing database
used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


In [11]:
# connectivity matrix
A = md.A

916428×916428 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  
* list the top 20 pages with the largest in-degrees?  
* histogram of out-degrees
* which the top 20 pages with the largest out-degrees?
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

In [12]:
# memory
Base.summarysize(A)

53276943

If converted to `Matrix{Float64}`, `A` will take $916428^2 \times 8 \approx 6.7 \cdot 10^{12}$ bytes of memory.

In [13]:
# number of web pages
size(A, 1)

916428

In [14]:
# number of edges
count(!iszero, A)

5105039

In [15]:
# number of dangling nodes
count(iszero, sum(A, dims = 2))

176974

In [16]:
using UnicodePlots
# histogram of in-degrees
histogram(log.(vec(sum(A, dims = 1)) .+ 1), nbins = 30)

              ┌                                        ┐ 
   [0.0, 0.5) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 201883            
   [0.5, 1.0) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 286895   
   [1.0, 1.5) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 173405                
   [1.5, 2.0) ┤▇▇▇▇▇▇▇▇▇▇ 88254                          
   [2.0, 2.5) ┤▇▇▇▇▇▇▇▇ 70956                            
   [2.5, 3.0) ┤▇▇▇▇▇ 49310                               
   [3.0, 3.5) ┤▇▇▇ 23073                                 
   [3.5, 4.0) ┤▇ 12001                                   
   [4.0, 4.5) ┤▇ 5880                                    
   [4.5, 5.0) ┤ 2813                                     
   [5.0, 5.5) ┤ 1049                                     
   [5.5, 6.0) ┤ 465                                      
   [6.0, 6.5) ┤ 233                                      
   [6.5, 7.0) ┤ 83                                       
   [7.0, 7.5) ┤ 37                                       
   [7.5, 8.0) ┤ 45                                       
   [8.0, 8.5) 

In [17]:
# top 20 pages with the largest in-degrees
sortperm(vec(sum(A, dims = 1)))[(end - 19):end]

20-element Vector{Int64}:
 687326
 173977
 384667
 765335
 213433
 407611
 459075
 558792
  41910
 551830
 828964
 605857
 819224
 163076
 885606
  32164
 751385
 504141
 597622
 537040

In [18]:
# histogram of out-degrees
histogram(log.(vec(sum(A, dims = 2)) .+ 1), nbins = 30)

              ┌                                        ┐ 
   [0.0, 0.2) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 176974   
   [0.2, 0.4) ┤ 0                                        
   [0.4, 0.6) ┤ 0                                        
   [0.6, 0.8) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 127937            
   [0.8, 1.0) ┤ 0                                        
   [1.0, 1.2) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 95253                   
   [1.2, 1.4) ┤▇▇▇▇▇▇▇▇▇▇▇▇ 65976                        
   [1.4, 1.6) ┤ 0                                        
   [1.6, 1.8) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 100331                 
   [1.8, 2.0) ┤▇▇▇▇▇▇▇ 40993                             
   [2.0, 2.2) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 75678                      
   [2.2, 2.4) ┤▇▇▇▇▇▇▇▇▇▇▇ 61762                         
   [2.4, 2.6) ┤▇▇▇▇▇▇▇▇▇ 48205                           
   [2.6, 2.8) ┤▇▇▇▇▇▇▇▇▇ 51304                           
   [2.8, 3.0) ┤▇▇▇▇▇▇▇▇▇ 47385                           
   [3.0, 3.2) ┤▇▇▇ 15646                                 
   [3.2, 3.4) 

In [19]:
# top 20 pages with the largest out-degrees
sortperm(vec(sum(A, dims = 2)))[(end - 19):end]

20-element Vector{Int64}:
 915274
 302734
 838279
 321092
 411594
 579315
 598189
 282141
 685696
 667585
 885729
 156951
 376429
 600480
 412411
 808644
 768092
 203749
 305230
 506743

In [20]:
# sparsity pattern
spy(A[1:10000, 1:10000])

                      Sparsity Pattern
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂⠀⠀⡄⢀⠀⠠

### Q2.4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

As noted above, if converted to `Matrix{Float64}`, `A` will take $916428^2 \times 8 \approx 6.7 \cdot 10^{12}$ bytes of memory. LU decomposition requires around $\approx \frac{2}{3}n^3$ flops. Assuming my computer can handel $10^{12}$ flops per second, it will take $\frac{2}{3} \cdot 916428^3 \times 10^{-12} \cdot (60 \cdot 60 \cdot 24)^{-1} \approx 6$ days. An eigen-solver will require even more flops.

### Q2.5 Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [21]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - Pᵗ in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA}  # adjacency matrix
    telep     :: T
    # working arrays
    r         :: Vector{T}
    x         :: Vector{T}
    y         :: Vector{T}
    storage_n :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n         = size(A, 1)
    r         = Float64.(vec(sum(A, dims = 2)))
    x         = fill(telep, n)
    y         = fill(1 - telep, n)
    for i in 1:n
        if r[i] == 0
            r[i] = Inf
            x[i] = 0
            y[i] = 1.0
        end
    end
    storage_n = Vector{T}(undef, size(A, 1))
    PageRankImPt(A, telep, r, x, y, storage_n)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
Base.getindex(M::PageRankImPt, i, j) = 1 - (M.x[j] * M.A[j, i] + M.y[j] / n)

# overwrite `out` by `(I - Pᵗ) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    M.storage_n .= v ./ M.r .* M.x
    mul!(out, transpose(M.A), M.storage_n)
    out .= v .- out .- dot(v, M.y) ./ size(M, 1)
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA <: Number, IA <: Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    mul!(out, M.A, v)
    out .= (out ./ M.r) .* M.x 
    out .= v .- out .- (sum(v) / size(M, 1)) .* M.y
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [23]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257-2021spring/ucla-biostat-257-2021spring.github.io/master/hw/hw3/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [24]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

@assert transpose(M) * ones(n) ≈ zeros(n)

In [25]:
norm(transpose(M) * ones(n) - zeros(n))

0.0

In [26]:
@assert M * xsol ≈ zeros(n)

LoadError: AssertionError: M * xsol ≈ zeros(n)

In [27]:
norm(M * xsol - zeros(n))

5.871226528840787e-16

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate little memory.

In [28]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup = (fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     19.727 ms (0.00% GC)
  median time:      23.326 ms (0.00% GC)
  mean time:        23.506 ms (0.00% GC)
  maximum time:     31.243 ms (0.00% GC)
  --------------
  samples:          204
  evals/sample:     1

In [29]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup = (fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     26.526 ms (0.00% GC)
  median time:      31.929 ms (0.00% GC)
  mean time:        32.957 ms (0.00% GC)
  maximum time:     47.048 ms (0.00% GC)
  --------------
  samples:          148
  evals/sample:     1

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [30]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are 30-40 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [31]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.378342822183319e-5, 2.07101553924505e-6, 3.663065985231059e-6, 7.527510785589858e-7, 8.633285997146752e-7, 1.7694182527348213e-6, 2.431230382907905e-7, 6.368417180736488e-7, 4.7449737037417485e-7, 2.689548611091868e-7  …  3.167946973972493e-6, 9.726880410215087e-7, 6.588614478531237e-7, 7.7370117747016e-7, 2.431230382907905e-7, 1.6219204214219128e-6, 3.9121300606247484e-7, 2.431230382907905e-7, 7.296033831312493e-6, 6.330939996667529e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821163900499833e-13,).
), time = 3.932726298, bytes = 1010081513, gctime = 0.07598819, gcstats = Base.GC_Diff(1010081513, 137, 0, 79560, 5, 0, 75988190, 3, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [32]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [33]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 7-8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [34]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[1.1364223365760337e-14 + 0.0im], Vector{ComplexF64}[[0.0056358269538073135 + 0.0im, 0.00034549143807829225 + 0.0im, 0.0006110808494125067 + 0.0im, 0.00012557561626034611 + 0.0im, 0.00014402240532802705 + 0.0im, 0.00029517830503983784 + 0.0im, 4.055832828677519e-5 + 0.0im, 0.00010623935784851675 + 0.0im, 7.915671116194688e-5 + 0.0im, 4.4867650667245184e-5 + 0.0im  …  0.0005284839899797019 + 0.0im, 0.00016226599147988575 + 0.0im, 0.00010991273837609196 + 0.0im, 0.00012907055855751131 + 0.0im, 4.055832828677519e-5 + 0.0im, 0.0002705723874320341 + 0.0im, 6.526302748332955e-5 + 0.0im, 4.055832828677519e-5 + 0.0im, 0.0012171406601327436 + 0.0im, 0.00010561415510698408 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.574093912491941e-14,).
), time = 9.251554612, bytes = 1586912888, gctime = 0.128523767, gcstats = Base.GC_Diff(1586912888, 213, 0, 163161, 91, 0, 128523767, 4, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [35]:
@assert abs(Real(vals[1])) < 1e-8

In [36]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [37]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 11-12 seconds.

### Q2.6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

In [39]:
sortperm(x_gmres)[(end - 19):end]

20-element Vector{Int64}:
 819224
 885606
 213433
   7315
 173977
 908352
 425771
 751385
 765335
 551830
 558792
  32164
 605857
 486981
 504141
 384667
 537040
 163076
  41910
 597622

In [40]:
sortperm(vec(sum(A, dims = 1)))[(end - 19):end]

20-element Vector{Int64}:
 687326
 173977
 384667
 765335
 213433
 407611
 459075
 558792
  41910
 551830
 828964
 605857
 819224
 163076
 885606
  32164
 751385
 504141
 597622
 537040

Top 20 pages from GMRES more or less match to the ranking according to in-degrees

### Q2.7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.